## 700 단어

In [ ]:
# Import
import warnings
import os
import pickle
import gc
from dotenv import load_dotenv
from konlpy.tag import Kkma
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_teddynote.retrievers import EnsembleRetriever, EnsembleMethod
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI

warnings.filterwarnings('ignore')
load_dotenv()

True

In [3]:
# Path
file_path = './data/700words.csv'
faiss_index_path = './index/700words_faiss_index.pkl'
bm25_index_path = './index/700words_bm25_index.pkl'

In [4]:
# Load & Documents
def load_csv_as_documents(file_path):
    df = pd.read_csv(file_path, encoding='utf-8')
    documents = []
    for i, row in df.iterrows():
        title = row['title'].strip() if pd.notna(row['title']) else ""
        content = row['content'].strip() if pd.notna(row['content']) else ""
        related_keyword = row['related_keyword'].strip() if pd.notna(row['related_keyword']) else ""
        
        combined_context = f"단어: {title}\n설명: {content}"
        
        doc = Document(
            page_content=combined_context,
            metadata={
                'title': title,
                'related_keyword': related_keyword,
                'doc_id': i
            }
        )
        documents.append(doc)
    return documents

In [5]:
documents = load_csv_as_documents(file_path)
gc.collect()

0

In [6]:
# check
print(f"총 {len(documents)}개의 문서가 로드되었습니다.")
print('--------------------------------------')
print(f"첫 번째 문서 내용: {documents[0].page_content[:100]}")
print('--------------------------------------')
print(f"두 번째 문서 내용: {documents[1].page_content[:100]}")
print('--------------------------------------')

총 700개의 문서가 로드되었습니다.
--------------------------------------
첫 번째 문서 내용: 단어: 가계부실위험지수(HDRI)
설명: 가구의 소득 흐름은 물론 금융 및 실물 자산까지 종합적으로 고려하여 가계부채의 부실위험을 평가하는 지표로, 가계의 채무상환능력을 소득 측면
--------------------------------------
두 번째 문서 내용: 단어: 가계수지
설명: 가정에서 일정 기간의 수입(명목소득)과 지출을 비교해서 남았는지 모자랐는지를 표시한 것을 가계수지(household's total income and exp
--------------------------------------


In [7]:
# kkma 한국어 형태소 분석기
kkma = Kkma()

def kkma_tokenize(text):
    return [token for token in kkma.morphs(text)]

In [8]:
# Embedding & Indexing
if os.path.exists(faiss_index_path) and os.path.exists(bm25_index_path):
    with open(faiss_index_path, 'rb') as f:
        faiss_index = pickle.load(f)
    with open(bm25_index_path, 'rb') as f:
        bm25_kkma = pickle.load(f)
else:
    embedding_model = HuggingFaceEmbeddings(
        model_name='paraphrase-multilingual-MiniLM-L12-v2',
        model_kwargs={'device': 'mps'},
        encode_kwargs={'normalize_embeddings': True},
    )

    faiss_index = FAISS.from_documents(documents, embedding_model).as_retriever(search_kwargs={"k": 5})
    with open(faiss_index_path, 'wb') as f:
        pickle.dump(faiss_index, f)
    
    bm25_kkma = BM25Retriever.from_documents(documents, preprocess_func=kkma_tokenize)
    bm25_kkma.k = 5
    with open(bm25_index_path, 'wb') as f:
        pickle.dump(bm25_kkma, f)

In [9]:
# Hybrid Search : CC method
weights = [0.93, 0.07]
hybrid_retriever = EnsembleRetriever(
    retrievers=[faiss_index, bm25_kkma],
    weights=weights,
    method=EnsembleMethod.CC
)

In [10]:
# KoReranker
model_path = "Dongjin-kr/ko-reranker"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# rerank
def rerank(query, retrieved_documents):
    pairs = [[query, doc.page_content] for doc in retrieved_documents]
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    with torch.no_grad():
        scores = model(**inputs, return_dict=True).logits.view(-1).float()
    reranked_docs = sorted(zip(retrieved_documents, scores), key=lambda x: x[1], reverse=True)
    return reranked_docs  # (documents, score) return

# optimize_context
def optimize_context(reranked_docs, max_tokens=6000):
    context = ""
    for doc, score in reranked_docs:
        if len(context) + len(doc.page_content) > max_tokens:
            break
        context += doc.page_content + "\n\n"
    return context.strip()

In [11]:
# Prompt
prompt_template = PromptTemplate(
    input_variables=["query", "retrieved_contents"],
    template="""
당신의 역할은 경제 용어에 대해 친절하고 쉽게 이해할 수 있는 설명을 제공하는 것입니다.
당신은 경제 지식이 없거나 경제 개념을 쉽게 배우고 싶은 사람들을 대상으로 '오늘의 단어' 포스팅을 작성합니다.

먼저 단어의 정의를 상세하게 설명해 주고, 일상 생활에 적용할 수 있는 관련 예시를 하나 간단하게 들어주세요.

마지막으로 이 용어를 이해하는 것이 왜 중요한지 요약하고 글을 마무리해 주세요.
경제 지식이 전혀 없는 사람도 쉽게 이해하고 흥미롭게 읽을 수 있도록 친근하고 쉽게 작성해 주세요.

# 주의사항:
1. 문서의 내용을 기반으로만 글을 작성하세요. 내용을 지어내거나 사실과 다르게 작성하지 마세요.
2. 만약 설명할 수 없는 부분이 있다면, '모르겠습니다'라고 답하세요.
3. 모든 제목은 #나 ## 같은 Markdown 표시 없이 굵은 글씨(**)로 나타나야 합니다. 예를 들어'## 경기는'는 '**경기**는'로 표시합니다.
4. 본문에는 일반 텍스트 형식을 사용하고, 필요할 경우 단어에만 굵은 글씨를 사용해주세요.
5. 제목을 제외하여 주세요.

이제 주제에 맞게 블로그 글을 작성해 주세요.
질문: {query}
단락: {retrieved_contents}
답변:
"""
)

In [12]:
# LLM & Chain
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
chain = LLMChain(llm=llm, prompt=prompt_template)

In [13]:
def generate_answer_with_debug(query, top_k_retrieve=5, top_k_rerank=3):
    # Step 1: Retrieve documents
    retrieved_documents = hybrid_retriever.invoke(query)
    
    # 상위 top_k_retrieve 문서만 유지
    unique_documents = []
    seen = set()
    for doc in retrieved_documents:
        if doc.page_content not in seen:
            unique_documents.append(doc)
            seen.add(doc.page_content)
        if len(unique_documents) == top_k_retrieve:
            break
    
    print("최종 Retrieve 단계에서 검색된 문서:")
    print("--------------------------------------------------")
    for i, doc in enumerate(unique_documents, 1):
        print(f"문서 {i}:\n내용: {doc.page_content[:100]}...\n")

    # Step 2: Rerank documents
    reranked_documents = rerank(query, unique_documents)
    reranked_documents = reranked_documents[:top_k_rerank]
    print("\nReranker를 통해 재정렬된 문서:")
    print("--------------------------------------------------")
    for i, (doc, score) in enumerate(reranked_documents, 1):
        print(f"문서 {i} (점수: {score}):\n내용: {doc.page_content[:100]}...\n")

    # Step 3: 최적화된 컨텍스트 생성 
    reranked_contents = optimize_context(reranked_documents)
    
    # Step 4: Generate response using LLM
    response = chain.run({"query": query, "retrieved_contents": reranked_contents})
    return response

In [14]:
query = "인플레이션은 무엇인가요?"
response = generate_answer_with_debug(query)

# Result
print("질문:", query)
print("답변:", response)

최종 Retrieve 단계에서 검색된 문서:
--------------------------------------------------
문서 1:
내용: 단어: 기대인플레이션
설명: 기대인플레이션은 향후 물가상승률에 대한 경제주체의 주관적인 전망을 나타내는 개념으로 물가안정을 추구하는 중앙은행이 관심을 기울이고 안정적으로 관리해야 ...

문서 2:
내용: 단어: 인플레이션
설명: 물가수준이 지속적으로 상승하는 현상을 인플레이션이라고 한다. 여기서 물가는 개별 상품의 가격을 평균하여 산출한 물가지수를 의미한다. 인플레이션은 물가상승 ...

문서 3:
내용: 단어: 근원인플레이션율
설명: 근원인플레이션율(core inflation rate)은 물가변동을 초래하는 여러 요인들 가운데 일시적인 공급충격의 영향을 제외한 기조적인 물가상승률을...

문서 4:
내용: 단어: 디플레이션
설명: 물가가 지속적으로 하락하는 현상을 말한다. 디플레이션(deflation) 하에서는 물가상 승률이 마이너스로 하락하는 인플레이션이 나타난다. 디플레이션이 발...

문서 5:
내용: 단어: 비용인상 인플레이션
설명: 재화나 서비스의 생산과 관련하여 투입요소의 비용 상승에 의해 물가가 지속적으로 상승하게 되는 것을 비용인상 인플레이션(cost-push infla...


Reranker를 통해 재정렬된 문서:
--------------------------------------------------
문서 1 (점수: 3.358750820159912):
내용: 단어: 인플레이션
설명: 물가수준이 지속적으로 상승하는 현상을 인플레이션이라고 한다. 여기서 물가는 개별 상품의 가격을 평균하여 산출한 물가지수를 의미한다. 인플레이션은 물가상승 ...

문서 2 (점수: -1.7418097257614136):
내용: 단어: 디플레이션
설명: 물가가 지속적으로 하락하는 현상을 말한다. 디플레이션(deflation) 하에서는 물가상 승률이 마이너스로 하락하는 인플레이션이 나타난다. 디플레이션이 발